In [ ]:
import os
import SimpleITK as sitk
import numpy as np

image_dir = "./image_data/images"
mask_dir = "./image_data/masks"
output_image_dir = "./image_data/output_224/images"
output_mask_dir = "./image_data/output_224/masks"
cache_dir = "./image_data/n4_cache"

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)
os.makedirs(cache_dir, exist_ok=True)

# ⚙️ 기준 spacing 및 shape (골반 전체 포함하도록 확대)
TARGET_SPACING = [1.0, 1.0, 1.0]
TARGET_SHAPE = (128, 224, 224)


# ✅ Bias field correction with caching
def bias_field_correction_cached(image, fname):
    cached_path = os.path.join(cache_dir, fname)
    if os.path.exists(cached_path):
        print(f"⚡ Using cached bias corrected: {fname}")
        return sitk.ReadImage(cached_path)

    image = sitk.Cast(image, sitk.sitkFloat32)
    mask = sitk.OtsuThreshold(image, 0, 1, 200)
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected = corrector.Execute(image, mask)
    sitk.WriteImage(corrected, cached_path)
    print(f"✅ Cached bias corrected: {fname}")
    return corrected

# ✅ Z-score normalization
def normalize_image(itk_image):
    image_np = sitk.GetArrayFromImage(itk_image).astype(np.float32)
    nonzero = image_np[image_np > 0]
    if nonzero.size == 0:
        return itk_image
    mean, std = np.mean(nonzero), np.std(nonzero)
    normalized = (image_np - mean) / (std + 1e-8)
    normalized[image_np == 0] = 0
    norm_image = sitk.GetImageFromArray(normalized.astype(np.float32))
    norm_image.CopyInformation(itk_image)
    return norm_image

# ✅ Resampling
def resample_image(itk_image, is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    new_size = [
        int(round(osz * ospc / tspc))
        for osz, ospc, tspc in zip(original_size, original_spacing, TARGET_SPACING)
    ]
    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear)
    resampler.SetOutputSpacing(TARGET_SPACING)
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(itk_image.GetDirection())
    resampler.SetOutputOrigin(itk_image.GetOrigin())
    resampler.SetDefaultPixelValue(0)
    return resampler.Execute(itk_image)

# ✅ Padding or cropping
def pad_or_crop(image_np, target_shape):
    current_shape = image_np.shape
    pad_width = [(0, max(t - c, 0)) for c, t in zip(current_shape, target_shape)]
    padded = np.pad(image_np, pad_width, mode='constant')
    cropped = padded[:target_shape[0], :target_shape[1], :target_shape[2]]
    return cropped

# 🔁 메인 루프
for fname in sorted(os.listdir(image_dir)):
    if not fname.endswith(".nii.gz") or fname in exclude_files:
        continue

    image_path = os.path.join(image_dir, fname)
    mask_path = os.path.join(mask_dir, fname)

    if not os.path.exists(mask_path):
        print(f"❌ Mask not found for {fname}")
        continue

    try:
        print(f"🔧 Processing: {fname}")

        # 1. Load
        image = sitk.ReadImage(image_path)
        mask = sitk.ReadImage(mask_path)

        # 2. Bias field correction (with cache)
        image = bias_field_correction_cached(image, fname)

        # 3. Z-score normalization
        image = normalize_image(image)

        # 4. Resample
        image_resampled = resample_image(image, is_label=False)
        mask_resampled = resample_image(mask, is_label=True)

        # 5. Pad or crop
        image_np = sitk.GetArrayFromImage(image_resampled).astype(np.float32)
        mask_np = sitk.GetArrayFromImage(mask_resampled).astype(np.uint8)

        image_fixed = pad_or_crop(image_np, TARGET_SHAPE)
        mask_fixed = pad_or_crop(mask_np, TARGET_SHAPE)

        # 6. Convert back to image
        image_out = sitk.GetImageFromArray(image_fixed)
        mask_out = sitk.GetImageFromArray(mask_fixed)

        # 7. Set metadata
        image_out.SetSpacing(TARGET_SPACING)
        image_out.SetOrigin(image_resampled.GetOrigin())
        image_out.SetDirection(image_resampled.GetDirection())

        mask_out.SetSpacing(TARGET_SPACING)
        mask_out.SetOrigin(image_resampled.GetOrigin())
        mask_out.SetDirection(image_resampled.GetDirection())

        # 8. Save
        sitk.WriteImage(image_out, os.path.join(output_image_dir, fname))
        sitk.WriteImage(mask_out, os.path.join(output_mask_dir, fname))
        print(f"✅ Done: {fname}")

    except Exception as e:
        print(f"❌ Failed {fname}: {e}")


🔧 Processing: P001_post_0000.nii.gz
⚡ Using cached bias corrected: P001_post_0000.nii.gz
✅ Done: P001_post_0000.nii.gz
🔧 Processing: P001_pre_0000.nii.gz
⚡ Using cached bias corrected: P001_pre_0000.nii.gz
✅ Done: P001_pre_0000.nii.gz
🔧 Processing: P002_post_0000.nii.gz
⚡ Using cached bias corrected: P002_post_0000.nii.gz
✅ Done: P002_post_0000.nii.gz
🔧 Processing: P002_pre_0000.nii.gz
⚡ Using cached bias corrected: P002_pre_0000.nii.gz
✅ Done: P002_pre_0000.nii.gz
🔧 Processing: P003_post_0000.nii.gz
⚡ Using cached bias corrected: P003_post_0000.nii.gz
✅ Done: P003_post_0000.nii.gz
🔧 Processing: P003_pre_0000.nii.gz
⚡ Using cached bias corrected: P003_pre_0000.nii.gz
✅ Done: P003_pre_0000.nii.gz
🔧 Processing: P004_post_0000.nii.gz
⚡ Using cached bias corrected: P004_post_0000.nii.gz
✅ Done: P004_post_0000.nii.gz
🔧 Processing: P004_pre_0000.nii.gz
⚡ Using cached bias corrected: P004_pre_0000.nii.gz
✅ Done: P004_pre_0000.nii.gz
🔧 Processing: P005_post_0000.nii.gz
⚡ Using cached bias corr

In [ ]:
import os
import SimpleITK as sitk
import numpy as np

image_dir = "./image_data/images"
mask_dir = "./image_data/masks"
output_image_dir = "./image_data/output_196/images"
output_mask_dir = "./image_data/output_196/masks"
cache_dir = "./image_data/n4_cache"

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)
os.makedirs(cache_dir, exist_ok=True)

# ⚙️ 기준 spacing 및 shape (골반 전체 포함하도록 확대)
TARGET_SPACING = [1.0, 1.0, 1.0]
TARGET_SHAPE = (128, 196, 196)


# ✅ Bias field correction with caching
def bias_field_correction_cached(image, fname):
    cached_path = os.path.join(cache_dir, fname)
    if os.path.exists(cached_path):
        print(f"⚡ Using cached bias corrected: {fname}")
        return sitk.ReadImage(cached_path)

    image = sitk.Cast(image, sitk.sitkFloat32)
    mask = sitk.OtsuThreshold(image, 0, 1, 200)
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrected = corrector.Execute(image, mask)
    sitk.WriteImage(corrected, cached_path)
    print(f"✅ Cached bias corrected: {fname}")
    return corrected

# ✅ Z-score normalization
def normalize_image(itk_image):
    image_np = sitk.GetArrayFromImage(itk_image).astype(np.float32)
    nonzero = image_np[image_np > 0]
    if nonzero.size == 0:
        return itk_image
    mean, std = np.mean(nonzero), np.std(nonzero)
    normalized = (image_np - mean) / (std + 1e-8)
    normalized[image_np == 0] = 0
    norm_image = sitk.GetImageFromArray(normalized.astype(np.float32))
    norm_image.CopyInformation(itk_image)
    return norm_image

# ✅ Resampling
def resample_image(itk_image, is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    new_size = [
        int(round(osz * ospc / tspc))
        for osz, ospc, tspc in zip(original_size, original_spacing, TARGET_SPACING)
    ]
    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear)
    resampler.SetOutputSpacing(TARGET_SPACING)
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(itk_image.GetDirection())
    resampler.SetOutputOrigin(itk_image.GetOrigin())
    resampler.SetDefaultPixelValue(0)
    return resampler.Execute(itk_image)

# ✅ Padding or cropping
def pad_or_crop(image_np, target_shape):
    current_shape = image_np.shape
    pad_width = [(0, max(t - c, 0)) for c, t in zip(current_shape, target_shape)]
    padded = np.pad(image_np, pad_width, mode='constant')
    cropped = padded[:target_shape[0], :target_shape[1], :target_shape[2]]
    return cropped

# 🔁 메인 루프
for fname in sorted(os.listdir(image_dir)):
    if not fname.endswith(".nii.gz") or fname in exclude_files:
        continue

    image_path = os.path.join(image_dir, fname)
    mask_path = os.path.join(mask_dir, fname)

    if not os.path.exists(mask_path):
        print(f"❌ Mask not found for {fname}")
        continue

    try:
        print(f"🔧 Processing: {fname}")

        # 1. Load
        image = sitk.ReadImage(image_path)
        mask = sitk.ReadImage(mask_path)

        # 2. Bias field correction (with cache)
        image = bias_field_correction_cached(image, fname)

        # 3. Z-score normalization
        image = normalize_image(image)

        # 4. Resample
        image_resampled = resample_image(image, is_label=False)
        mask_resampled = resample_image(mask, is_label=True)

        # 5. Pad or crop
        image_np = sitk.GetArrayFromImage(image_resampled).astype(np.float32)
        mask_np = sitk.GetArrayFromImage(mask_resampled).astype(np.uint8)

        image_fixed = pad_or_crop(image_np, TARGET_SHAPE)
        mask_fixed = pad_or_crop(mask_np, TARGET_SHAPE)

        # 6. Convert back to image
        image_out = sitk.GetImageFromArray(image_fixed)
        mask_out = sitk.GetImageFromArray(mask_fixed)

        # 7. Set metadata
        image_out.SetSpacing(TARGET_SPACING)
        image_out.SetOrigin(image_resampled.GetOrigin())
        image_out.SetDirection(image_resampled.GetDirection())

        mask_out.SetSpacing(TARGET_SPACING)
        mask_out.SetOrigin(image_resampled.GetOrigin())
        mask_out.SetDirection(image_resampled.GetDirection())

        # 8. Save
        sitk.WriteImage(image_out, os.path.join(output_image_dir, fname))
        sitk.WriteImage(mask_out, os.path.join(output_mask_dir, fname))
        print(f"✅ Done: {fname}")

    except Exception as e:
        print(f"❌ Failed {fname}: {e}")


단계	전처리 내용	설명
1️⃣	Bias Field Correction	N4 알고리즘으로 intensity non-uniformity 제거 캐싱추가
2️⃣	Intensity Clipping	이상치 제거 (0.5~99.5 percentile)
3️⃣	Z-score Normalization	background 제외하고 평균 0, 표준편차 1로 정규화
4️⃣	Resampling	spacing → [1.0, 1.0, 1.0], 마스크는 NN
5️⃣	Pad or Crop	shape 
6️⃣	Affine 정보 유지	origin, spacing, direction 포함